In [1]:
from pyrtl import *
import pyrtl

### Now, we can define states. 


<ol>
    <li><b>State Name:</b><code>q0</code>, <b>Outputs:</b> <code>out_0 = 0</code>, <code>out_1 = a ^ b</code></li>
    <li><b>State Name:</b><code>q1</code>, <b>Outputs:</b> <code>out_0 = a & b & c</code>, <code>out_1 = 1</code></li>
    <li><b>State Name:</b><code>q2</code>, <b>Outputs:</b> <code>out_0 = b | c</code>, <code>out_1 = b & c</code></li>
    <li><b>State Name:</b><code>q3</code>, <b>Outputs:</b> <code>out_0 = 1</code>, <code>out_1 = a | b</code></li>
    <li><b>State Name:</b><code>q4</code>, <b>Outputs:</b> <code>out_0 = 0</code>, <code>out_1 = a | b | c</code></li>
</ol>

<img src = "https://yliu47-nd.neocities.org/%E6%88%AA%E5%B1%8F2022-12-07%20%E4%B8%8B%E5%8D%883.33.30.png" width=700 height=700>

<img src = "https://yliu47-nd.neocities.org/%E6%88%AA%E5%B1%8F2022-12-07%20%E4%B8%8B%E5%8D%883.43.52.png" width=700 height=700>

### Test Suite

Here is the truth table for our Finite State Machine

| State | out_0 | out_1 | next_state 0 | next_state 1 |
|---|---|---|---|---|
|q0| 0 | a ^ b | q0 | q1 |
|q1| a & b & c | 1 | q4 | q2 |
|q2| b or c | b & c | q0 | q1 |
|q3| 1 | a or b| q3 | q2 |
|q4| 0 | a or b or c |q0 | q3 |

Based on these functions and the FSM Diagram, here are the <i>expected</i> outputs:


| Start State | a | b | c | out_0 | out_1 | Control | Next State |
|---|---|---|---|---|---|---|---|
|<b><font color="purple">q0</font></b>|1|1|1|<b><font color="red">0</font></b>| a ^ b = <b><font color="blue">1</font></b>| 1 | q1 |
|<b><font color="purple">q1</font></b>|1|0|0|a & b & c = <b><font color="red">0</font></b>| <b><font color="blue">1</font></b> | 0 | q4 |
|<b><font color="purple">q4</font></b>|1|1|1|<b><font color="red">0</font></b>| a & c =<b><font color="blue">1</font></b> | 1 | q3 |
|<b><font color="purple">q3</font></b>|0|1|0|<b><font color="red">1</font></b>| a ^ b =<b><font color="blue">1</font></b> | 0 | q3 |
|<b><font color="purple">q3</font></b>|0|1|1|<b><font color="red">1</font></b>| a ^ b = <b><font color="blue">1</font></b> | 1 | q2 |
|<b><font color="purple">q2</font></b>|1|0|0|a & b & c =<b><font color="red">0</font></b>| b & c =<b><font color="blue">0</font></b> | 1 | q1 |<b><font color="purple">

### Additional Functions

Click through these functions that we will use inside each state:

In [2]:
def two_input_and(a_input, b_input):
    y_output = a_input & b_input
    
    return y_output

In [3]:
def two_input_or(a_input, b_input):
    y_output = a_input | b_input
    
    return y_output

In [4]:
def two_input_xor(a_input, b_input):
    y_output = a_input ^ b_input
    
    return y_output

In [5]:
def three_input_and( a_input, b_input, c_input ):
    
    y_output = a_input & b_input & c_input
    
    return y_output

In [6]:
def three_input_or( a_input, b_input, c_input ):

    y_output = a_input | b_input | c_input

    return y_output

### First, define each state in the FSM

To promote modularity, and reduce the challenges of troubleshooting, keep the "current state operations" and the "next state calculations separate.

In [7]:
def curr_state_op( curr_state, a_input, b_input, c_input ):
    
    with pyrtl.conditional_assignment:

        state_out_0 = pyrtl.WireVector(1)
        state_out_1 = pyrtl.WireVector(1)
        

        with curr_state == 0:
            state_out_0 |= 0
            state_out_1 |= two_input_xor( a_input, b_input )
        
        with curr_state == 1:
            state_out_0 |= three_input_and( a_input, b_input, c_input )
            state_out_1 |= 1
        
        with curr_state == 2:
            state_out_0 |= two_input_or(b_input, c_input )
            state_out_1 |= two_input_and(b_input, c_input)

        with curr_state == 3:
            state_out_0 |= 1
            state_out_1 |= two_input_or(a_input, b_input)

        with curr_state == 4:
            state_out_0 |= 0
            state_out_1 |= three_input_or(a_input, b_input, c_input)
            
        return state_out_0, state_out_1

In [8]:
def next_state( curr_state, control_signal ):
    with pyrtl.conditional_assignment:
            

        with curr_state == 0:
            
            with control_signal == 0:
                curr_state.next |= 0
                
            with control_signal == 1:
                curr_state.next |= 1

                
        with curr_state == 1:
            
            with control_signal == 0:
                curr_state.next |= 4
                
            with control_signal == 1:
                curr_state.next |= 2
                

        with curr_state == 2:
            
            with control_signal == 0:
                curr_state.next |= 0
                
            with control_signal == 1:
                curr_state.next |= 1
                

        with curr_state == 3:
            
            with control_signal == 0:
                curr_state.next |= 3
                
            with control_signal == 1:
                curr_state.next |= 2
                


        with curr_state == 4:
            
            with control_signal == 0:
                curr_state.next |= 0
                
            with control_signal == 1:
                curr_state.next |= 3
        

        return curr_state.next

In [9]:
def AIgame_fsm_simulate( ):
    
    pyrtl.reset_working_block()
    
    control_signal = pyrtl.Input(2, 'control_signal')
    a_in = pyrtl.Input(1, 'a_in')
    b_in = pyrtl.Input(1, 'b_in')
    c_in = pyrtl.Input(1, 'c_in')
    
    output_0 = pyrtl.Output(1, 'output_0')
    output_1 = pyrtl.Output(1, 'output_1')
    
    curr_state = pyrtl.Register(3, 'curr_state')

    inter_0, inter_1 = curr_state_op( curr_state, a_in, b_in, c_in )
    
    output_0 <<= inter_0
    output_1 <<= inter_1
    
    curr_state = next_state( curr_state, control_signal )
    
    sim = pyrtl.Simulation()
    
    control_signals = [1,1,0,1,0,0,0,1,1]
    a_inputs        = [1,1,1,0,0,1,1,1,1]
    b_inputs        = [1,0,1,1,1,0,0,1,1]
    c_inputs        = [1,0,1,0,1,0,0,0,1]
    

    
    for value in range(0, len(a_inputs)):

        sim.step({
            
            'control_signal' : control_signals[value],
            'a_in' : a_inputs[value],
            'b_in' : b_inputs[value],
            'c_in' : c_inputs[value] 
            
        })
    
    sim.tracer.render_trace()

In [10]:
AIgame_fsm_simulate()

<IPython.core.display.Javascript object>

### Check: Compare the generated waveform with the table we developed earlier

| Start State | a | b | c | out_0 | out_1 | Control | Next State |
|---|---|---|---|---|---|---|---|
|<b><font color="purple">q0</font></b>|1|1|1|<b><font color="red">0</font></b>| a ^ b = <b><font color="blue">1</font></b>| 1 | q1 |
|<b><font color="purple">q1</font></b>|1|0|0|a & b & c = <b><font color="red">0</font></b>| <b><font color="blue">1</font></b> | 0 | q4 |
|<b><font color="purple">q4</font></b>|1|1|1|<b><font color="red">0</font></b>| a & c =<b><font color="blue">1</font></b> | 1 | q3 |
|<b><font color="purple">q3</font></b>|0|1|0|<b><font color="red">1</font></b>| a ^ b =<b><font color="blue">1</font></b> | 0 | q3 |
|<b><font color="purple">q3</font></b>|0|1|1|<b><font color="red">1</font></b>| a ^ b = <b><font color="blue">1</font></b> | 1 | q2 |
|<b><font color="purple">q2</font></b>|1|0|0|a & b & c =<b><font color="red">0</font></b>| b & c =<b><font color="blue">0</font></b> | 1 | q1 |<b><font color="purple">